# MODEL TRAINING - SET 2

### 1.1 Import Data and Required Packages
#### Importing Pandas, Numpy, Matplotlib, Seaborn and Warnings Library.

In [370]:
#Importing recquired packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

##### Importing Modelling Libraries

In [371]:
#Importing ML Packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.multioutput import RegressorChain

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.cross_decomposition import PLSRegression 
from sklearn.svm import SVR                          
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neighbors import KNeighborsRegressor   

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor

from xgboost import XGBRegressor

#Importing DL Packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
import datetime

#### Importing CSV data as Pandas dataframe

In [372]:
#Importing CSV data as Pandas dataframe
df=pd.read_csv('dataset/Extracted_CrudeData.csv')

#Show top 5 rows of the dataframe
df.head()

,Crude Name,StdLiquidDensity (kg/m3),SulfurByWt (%),ConradsonCarbonByWt (%),NitrogenByWt (%),Distillation Mass @ X Pct (C)@ 1 (%) - TBP,Distillation Mass @ X Pct (C)@ 5 (%) - TBP,Distillation Mass @ X Pct (C)@ 10 (%) - TBP,Distillation Mass @ X Pct (C)@ 30 (%) - TBP,Distillation Mass @ X Pct (C)@ 50 (%) - TBP,...,AromByWt (%),NaphthenesByWt (%),ParaffinsByWt (%),KinematicViscosity (cSt)@ 37.78 (C),KinematicViscosity (cSt)@ 98.89 (C),KinematicViscosity (cSt)@ 20 (C),KinematicViscosity (cSt)@ 40 (C),KinematicViscosity (cSt)@ 50 (C),KinematicViscosity (cSt)@ 100 (C),KinematicViscosity (cSt)@ 150 (C)
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,80.470529,15.097104,4.432367,57.2992918615153,7.64442822441154,138.700886,52.450950,34.581349,7.453127,NaN
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,24.571141,41.011754,34.417105,1.7573028543796,0.833437569080932,2.091216,1.700584,1.443025,0.824673,NaN
2,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,45.113632,53.390135,1.496234,208.950655083803,7.7853062269412,1077.054144,178.565076,53.097085,NaN,NaN
3,Alba-2002,932.299296,1.188621,4.615622,0.226565,123.671482,208.861673,254.256413,357.502064,459.755466,...,43.692823,54.136415,2.170762,57.5107882851462,7.38119815238387,NaN,52.597843,34.537234,7.178422,NaN
4,Alba-2012,934.718850,1.254322,7.296455,0.220453,127.209059,191.668726,241.194850,350.718785,450.253389,...,36.755820,60.660536,2.583644,74.735481797634,8.27927920440891,NaN,67.443091,41.725218,8.042583,NaN


In [373]:
#Shape of dataset
df.shape

(114, 24)

In [374]:
#Renaming columns for better readability
df.rename(columns={
    'Crude Name': 'Crude_Name',
    'StdLiquidDensity (kg/m3)': 'StdLiqDensity_kgm3',
    'SulfurByWt (%)': 'SulfurWt_pct',
    'ConradsonCarbonByWt (%)': 'ConradsonCarbon_wt_pct',
    'NitrogenByWt (%)': 'NitrogenWt_pct',
    
    'Distillation Mass @ X Pct (C)@ 1 (%) - TBP': 'TBP_TempAt_1pct',
    'Distillation Mass @ X Pct (C)@ 5 (%) - TBP': 'TBP_TempAt_5pct',
    'Distillation Mass @ X Pct (C)@ 10 (%) - TBP': 'TBP_TempAt_10pct',
    'Distillation Mass @ X Pct (C)@ 30 (%) - TBP': 'TBP_TempAt_30pct',
    'Distillation Mass @ X Pct (C)@ 50 (%) - TBP': 'TBP_TempAt_50pct',
    'Distillation Mass @ X Pct (C)@ 70 (%) - TBP': 'TBP_TempAt_70pct',
    'Distillation Mass @ X Pct (C)@ 90 (%) - TBP': 'TBP_TempAt_90pct',
    'Distillation Mass @ X Pct (C)@ 95 (%) - TBP': 'TBP_TempAt_95pct',
    'Distillation Mass @ X Pct (C)@ 99 (%) - TBP': 'TBP_TempAt_99pct',

    'AromByWt (%)': 'AromWt_pct',
    'NaphthenesByWt (%)': 'NaphWt_pct',
    'ParaffinsByWt (%)': 'ParaWt_pct',

    'KinematicViscosity (cSt)@ 37.78 (C)': 'KV_37.78c_cSt',
    'KinematicViscosity (cSt)@ 98.89 (C)': 'KV_98.89c_cSt',
    'KinematicViscosity (cSt)@ 20 (C)': 'KV_20c_cSt',
    'KinematicViscosity (cSt)@ 40 (C)': 'KV_40c_cSt',
    'KinematicViscosity (cSt)@ 50 (C)': 'KV_50c_cSt',
    'KinematicViscosity (cSt)@ 100 (C)': 'KV_100c_cSt',
    'KinematicViscosity (cSt)@ 150 (C)': 'KV_150c_cSt'
}, inplace=True)
df.head()


,Crude_Name,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,...,AromWt_pct,NaphWt_pct,ParaWt_pct,KV_37.78c_cSt,KV_98.89c_cSt,KV_20c_cSt,KV_40c_cSt,KV_50c_cSt,KV_100c_cSt,KV_150c_cSt
0,Achinsk-2015,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,...,80.470529,15.097104,4.432367,57.2992918615153,7.64442822441154,138.700886,52.450950,34.581349,7.453127,NaN
1,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,24.571141,41.011754,34.417105,1.7573028543796,0.833437569080932,2.091216,1.700584,1.443025,0.824673,NaN
2,Alba-1994,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,...,45.113632,53.390135,1.496234,208.950655083803,7.7853062269412,1077.054144,178.565076,53.097085,NaN,NaN
3,Alba-2002,932.299296,1.188621,4.615622,0.226565,123.671482,208.861673,254.256413,357.502064,459.755466,...,43.692823,54.136415,2.170762,57.5107882851462,7.38119815238387,NaN,52.597843,34.537234,7.178422,NaN
4,Alba-2012,934.718850,1.254322,7.296455,0.220453,127.209059,191.668726,241.194850,350.718785,450.253389,...,36.755820,60.660536,2.583644,74.735481797634,8.27927920440891,NaN,67.443091,41.725218,8.042583,NaN


#### Duplicate Values, Missing Values & Data Types

In [375]:
#Exploring data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Crude_Name              114 non-null    object 
 1   StdLiqDensity_kgm3      114 non-null    float64
 2   SulfurWt_pct            114 non-null    float64
 3   ConradsonCarbon_wt_pct  114 non-null    float64
 4   NitrogenWt_pct          114 non-null    float64
 5   TBP_TempAt_1pct         114 non-null    float64
 6   TBP_TempAt_5pct         114 non-null    float64
 7   TBP_TempAt_10pct        114 non-null    float64
 8   TBP_TempAt_30pct        114 non-null    float64
 9   TBP_TempAt_50pct        114 non-null    float64
 10  TBP_TempAt_70pct        114 non-null    float64
 11  TBP_TempAt_90pct        114 non-null    float64
 12  TBP_TempAt_95pct        114 non-null    float64
 13  TBP_TempAt_99pct        114 non-null    float64
 14  AromWt_pct              114 non-null    fl

In [376]:
#Changing data types
df["KV_37.78c_cSt"] = pd.to_numeric(df["KV_37.78c_cSt"], errors="coerce")
df["KV_98.89c_cSt"] = pd.to_numeric(df["KV_98.89c_cSt"], errors="coerce")

In [377]:
#Checks for duplicate values
df.duplicated().sum()

np.int64(0)

In [378]:
#Checks for missing values
df.isna().sum()

Crude_Name                  0
StdLiqDensity_kgm3          0
SulfurWt_pct                0
ConradsonCarbon_wt_pct      0
NitrogenWt_pct              0
TBP_TempAt_1pct             0
TBP_TempAt_5pct             0
TBP_TempAt_10pct            0
TBP_TempAt_30pct            0
TBP_TempAt_50pct            0
TBP_TempAt_70pct            0
TBP_TempAt_90pct            0
TBP_TempAt_95pct            0
TBP_TempAt_99pct            0
AromWt_pct                  0
NaphWt_pct                  0
ParaWt_pct                  0
KV_37.78c_cSt               5
KV_98.89c_cSt               2
KV_20c_cSt                 46
KV_40c_cSt                 38
KV_50c_cSt                 22
KV_100c_cSt                32
KV_150c_cSt               101
dtype: int64

In [379]:
#Deleting columns with more than 50% missing values
df.drop(columns='KV_150c_cSt', inplace=True)
df.shape

(114, 23)

In [380]:
df.loc[df['Crude_Name'] == 'Cinta-1983', 'KV_37.78c_cSt'] = 14.183399

df = df.dropna(subset=['KV_37.78c_cSt', 'KV_98.89c_cSt'])


#### ML Model Training

In [381]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt', 
                    'KV_20c_cSt', 'KV_40c_cSt', 'KV_50c_cSt', 'KV_100c_cSt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590
3,932.299296,1.188621,4.615622,0.226565,123.671482,208.861673,254.256413,357.502064,459.755466,566.808669,718.285795,790.386373,925.151275
4,934.718850,1.254322,7.296455,0.220453,127.209059,191.668726,241.194850,350.718785,450.253389,568.009018,737.938988,819.140733,963.333194


In [382]:
#Dependent Features
y = np.log1p(df[['KV_37.78c_cSt', 'KV_98.89c_cSt']]) 

In [383]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Train size: 88, Test size: 22


((88, 13), (22, 13), (88, 2), (22, 2))

In [384]:
#Define categorical and numerical feature columns
num_features = [col for col in X.columns]

#Define transformations for each feature
numeric_transformer = StandardScaler()                    

#Apply scaling to numeric features
preprocessor = ColumnTransformer([("StandardScaler", numeric_transformer, num_features)])


In [385]:
#Machine Learning Algorithms
models = {
    "Linear Regression": Pipeline([
        ('pre', preprocessor),
        ('linreg', MultiOutputRegressor(LinearRegression()))
    ]),
    "Lasso Regression": Pipeline([
        ('pre', preprocessor),
        ('lasso', MultiOutputRegressor(Lasso(alpha=1.0)))
    ]),
    "Ridge Regression": Pipeline([
        ('pre', preprocessor),
        ('ridge', MultiOutputRegressor(Ridge(alpha=1.0)))
    ]),
    "ElasticNet": Pipeline([
        ('pre', preprocessor),
        
        ('elasticnet', MultiOutputRegressor(ElasticNet(alpha=1.0)))
    ]),
    "SVR": Pipeline([
        ('pre', preprocessor),
        
        ('svr', MultiOutputRegressor(SVR(kernel='rbf', C=100, epsilon=0.1)))
    ]),
    "KNN": Pipeline([
        ('pre', preprocessor), 
        ('knn', MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3)))
    ]),
    "Decision Tree Regressor": Pipeline([
        ('pre', preprocessor),  
        ('dt', MultiOutputRegressor(DecisionTreeRegressor(random_state=42)))
    ]),
    "Extra Trees Regressor": Pipeline([
        ('pre', preprocessor),  
        ('et', MultiOutputRegressor(ExtraTreesRegressor(n_estimators=100, random_state=42)))
    ]),
    "Random Forest": Pipeline([
        ('pre', preprocessor),
        ('rf', MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42)))
    ]),
    "Gradient Boosting": Pipeline([
        ('pre', preprocessor),
        ('gb', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)))
    ]),
    "Adaboost": Pipeline([
        ('pre', preprocessor),
        ('ab', MultiOutputRegressor(AdaBoostRegressor(n_estimators=100, random_state=42)))
    ]),
    "XGBoost": Pipeline([
        ('pre', preprocessor),
        ('xgb', MultiOutputRegressor(XGBRegressor(n_estimators=50, max_depth=3, random_state=42)))
    ]),
    
}

In [386]:
#Train and evaluate all models
print("Training ML Models...")

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'R2': r2_score(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
    }
results_df = pd.DataFrame(results).T
print(results_df.sort_values('R2', ascending=False))


Training ML Models...
                               R2       MAE      RMSE
XGBoost                  0.903803  0.307854  0.436866
Linear Regression        0.877258  0.374019  0.530134
Random Forest            0.873777  0.326601  0.504670
Gradient Boosting        0.870563  0.324482  0.543603
Adaboost                 0.864273  0.374757  0.531423
Ridge Regression         0.864002  0.387683  0.569866
Extra Trees Regressor    0.863860  0.323057  0.557194
KNN                      0.862218  0.339080  0.521988
SVR                      0.856159  0.436188  0.620274
Decision Tree Regressor  0.802495  0.411760  0.581175
ElasticNet               0.755401  0.588809  0.708219
Lasso Regression         0.397445  0.843874  0.991088


#### ANN Model Training

In [387]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt', 
                    'KV_20c_cSt', 'KV_40c_cSt', 'KV_50c_cSt', 'KV_100c_cSt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,926.440713,24.606786,5.827369,0.209874,-17.813183,99.504433,239.896662,398.590562,481.984170,561.500466,665.837149,714.976170,806.217469
1,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
2,937.762899,1.322111,5.847739,0.196515,159.292817,225.625562,264.407311,363.170601,470.497570,595.448493,779.746511,866.765919,1005.918590
3,932.299296,1.188621,4.615622,0.226565,123.671482,208.861673,254.256413,357.502064,459.755466,566.808669,718.285795,790.386373,925.151275
4,934.718850,1.254322,7.296455,0.220453,127.209059,191.668726,241.194850,350.718785,450.253389,568.009018,737.938988,819.140733,963.333194


In [388]:
#Dependent Features
y = df[['AromWt_pct', 'NaphWt_pct', 'ParaWt_pct']]

In [389]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((88, 13), (22, 13), (88, 3), (22, 3))

In [390]:
scaler_X = StandardScaler()
scaler_y = StandardScaler() 

#Fit Train & transform Test
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled  = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled  = scaler_y.transform(y_test)

In [391]:
pca=PCA(n_components=0.95)
X_train_pca=pca.fit_transform(X_train_scaled)
X_test_pca=pca.transform(X_test_scaled)

In [392]:
ann_model = Sequential([Dense(64, activation='relu', input_shape=(X_train_pca.shape[1],)),
                    
                    Dense(32, activation='relu'),
                    Dense(3, activation='linear')])
ann_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499 (9.76 KB)

 Trainable params: 2,499 (9.76 KB)

 Non-trainable params: 0 (0.00 B)

In [393]:
#Compile the model
ann_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [394]:
#Setup Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [395]:
#Setup Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

In [400]:
print("Training Optimized ANN...")
history = ann_model.fit(X_train_pca, y_train_scaled, validation_data=(X_test_pca, y_test_scaled), epochs=500, batch_size=8,
                  callbacks=[early_stopping, tensorboard_callback])

Training Optimized ANN...
Epoch 1/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.4535 - mae: 0.4808 - val_loss: 0.8179 - val_mae: 0.6539
Epoch 2/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.4454 - mae: 0.4737 - val_loss: 0.8244 - val_mae: 0.6486
Epoch 3/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.4333 - mae: 0.4670 - val_loss: 0.8317 - val_mae: 0.6491
Epoch 4/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.4220 - mae: 0.4621 - val_loss: 0.8359 - val_mae: 0.6475
Epoch 5/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.4104 - mae: 0.4571 - val_loss: 0.8337 - val_mae: 0.6474
Epoch 6/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.4030 - mae: 0.4546 - val_loss: 0.8389 - val_mae: 0.6506
Epoch 7/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3948 - mae: 0.4488 - val_loss: 0.8535 - val_mae: 0.6465
Epoch 8/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.3930 - mae: 0.4455 - val_loss: 0.8352 - val_mae: 0.6441
Epoch 9/500
11/11 ━━━━━━━━━━━━

In [401]:
print("Predicting on Test Set...")
y_pred_scaled = ann_model.predict(X_test_pca)

#Convert back to real percentages
y_pred_real = scaler_y.inverse_transform(y_pred_scaled)
y_test_real = scaler_y.inverse_transform(y_test_scaled)

y_pred_real = 100 * y_pred_real / np.sum(y_pred_real, axis=1, keepdims=True)

#Calculate Metrics on the Real Data
r2 = r2_score(y_test_real, y_pred_real, multioutput='uniform_average')
mae = mean_absolute_error(y_test_real, y_pred_real)
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_real))

print("ANN PERFORMANCE REPORT")
print(f"R2 Score: {r2:.4f}")
print(f"MAE:      {mae:.4f} %")
print(f"RMSE:     {rmse:.4f} %")
print("-" * 100)

results_df = pd.DataFrame(
    data=np.hstack((y_test_real, y_pred_real)),
    columns=['Actual_Arom', 'Actual_Naph', 'Actual_Para', 'Pred_Arom', 'Pred_Naph', 'Pred_Para']
)
print("\nSample Predictions (First 5 Rows):")
display(results_df.head())

Predicting on Test Set...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
ANN PERFORMANCE REPORT
R2 Score: 0.0947
MAE:      11.3188 %
RMSE:     16.1121 %
----------------------------------------------------------------------------------------------------

Sample Predictions (First 5 Rows):


,Actual_Arom,Actual_Naph,Actual_Para,Pred_Arom,Pred_Naph,Pred_Para
0,60.675615,31.968498,7.355888,59.593472,33.402195,7.004331
1,20.747572,61.569046,17.683382,34.929756,46.480808,18.589436
2,36.755820,60.660536,2.583644,48.273506,44.718151,7.008342
3,58.067947,28.688084,13.243970,68.598618,23.637199,7.764177
4,42.651632,53.040355,4.308013,45.975414,45.926540,8.098054
